In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython import display
import copy
from PIL import Image
%matplotlib inline

Zadanie 1

In [ ]:
def arbitrary_swap(T, vector):
    swaps = int(len(vector)*T)
    for i in range(swaps):
        j = np.random.randint(len(vector))
        k = np.random.randint(len(vector))
        vector[[j, k],:] = vector[[k, j],:]
    return vector

In [ ]:
def consecutive_swap(T, vector):
    swaps = int(len(vector)*T)
    for i in range(swaps):
        k = np.random.randint(len(vector))
        vector[[k-1, k],:] = vector[[k, k-1],:]
    return vector

In [ ]:
def uniform_points(n):
    return np.random.random((n,2))

In [ ]:
def normal_points(n,parts):
    n//=parts
    n*=parts
    #liczba punktów n musi być podzielna przez liczbę częsci
    sqrt =int(np.sqrt(parts))
    centres = np.array([[[parts*i,parts*j] for i in range(sqrt)]for j in range(sqrt)]).reshape((parts,2))
    result =np.array([[np.random.normal(loc = i, scale = 0.1) for j in range(n//parts)]for i in centres]).reshape((n,2))
    np.random.shuffle(result)
    return result

In [ ]:
def length(points):
    simple_points = [(points[i][0],points[i][1]) for i in range(len(points))]
    vectors = [(points[i][0]-points[i-1][0],points[i][1]-points[i-1][1]) for i in range(1,len(points))]
    return sum([i[1]**2+i[0]**2 for i in vectors])

In [ ]:
points = uniform_points(100)

In [ ]:
def annealing(points,swap,Tfunc = lambda x: 0.95*x,ax=None):
    best = points.copy()
    best_len = length(best)
    states = [best.copy()]
    T =1
    lengths =[best_len]
    no_change_counter=0
    i=1
    while T > 0.005 or no_change_counter <0.05*i:
        #jeżeli 20 razy nie uzyskam lepszej wartości, to kończę pętlę
        points = best.copy()
        swap(T, points)
        p_len = length(points) 
        lengths.append(p_len.copy())
        states.append(points.copy())
        if p_len< best_len:
            best_len = p_len
            best = points.copy()
            no_change_counter =0
        else:
            no_change_counter +=1
        T = Tfunc(T)
        i += 1
    if ax == None:
        plt.plot([i for i in range(len(lengths))],lengths)
        plt.show()
    else:
        ax.plot([i for i in range(len(lengths))],lengths)
    return best

In [ ]:
def animate(states):
    def AnimationFunction(frame):
        line_plotted.set_data((frame[:,0],frame[:,1]))
        plt.xlim([np.min(states[0][:,0]),np.max(states[0][:,0])])
        plt.ylim([np.min(states[0][:,1]),np.max(states[0][:,1])])
        return line_plotted
        # line is set with new values of x and y
    Figure = plt.figure()
    lines_plotted = plt.plot([])
    line_plotted = lines_plotted[0]
    step = len(states)//150+1
    anim_created = animation.FuncAnimation(Figure, AnimationFunction, frames=states[0::step],blit=False)
    video = anim_created.to_html5_video()
    html = display.HTML(video)
    display.display(html)
    plt.show()

In [ ]:
def visualize(states,best):
    plt.scatter(states[0][:,0],states[0][:,1])
    plt.show()
    animate(states)

In [ ]:
def draw_annealing(points,swap,Tfunc = lambda x: 0.95*x,ax = None):
    best = points.copy()
    best_len = length(best)
    states = [best.copy()]
    T =0.95
    lengths =[best_len]
    no_change_counter=0
    i=1
    while T > 0.005 or no_change_counter <0.05*i:
        #jeżeli 20 razy nie uzyskam lepszej wartości, to kończę pętlę
        points = best.copy()
        swap(T, points)
        p_len = length(points) 
        lengths.append(p_len.copy())
        states.append(points.copy())
        if p_len< best_len:
            best_len = p_len
            best = points.copy()
            no_change_counter =0
        else:
            no_change_counter +=1
        T = Tfunc(T)
        i+=1
    visualize(states,best)
    plt.plot([i for i in range(len(lengths))],lengths)
    plt.show()
    return best

In [ ]:
#Generowanie wszystkich filmów i wykresów może potrwać długo
sizes = [100,500,1000]
distributions = [lambda n : uniform_points(n), lambda n: normal_points(n,4), lambda n: normal_points(n,9)]
for size in sizes:
    print(size)
    for dist in distributions:
        points = dist(size)
        draw_annealing(points,arbitrary_swap,lambda x: 0.999*x)

In [ ]:
print("T(n) = 0.99*T(n-1)")
sizes = [100,500,1000]
distributions = [lambda n : uniform_points(n), lambda n: normal_points(n,4), lambda n: normal_points(n,9)]
for size in sizes:
    for dist in distributions:
        points = dist(size)
        fig,ax =plt.subplots(1,2)
        print(size)
        arb = annealing(points.copy(),arbitrary_swap,lambda x: 0.99*x,ax[0])
        con = annealing(points.copy(),consecutive_swap,lambda x: 0.99*x,ax[1])
        print("arbitrary swap result:\t\t",length(arb))
        print("consecutive swap result:\t",length(con))
        ax[0].set_title('arbitrary')
        ax[1].set_title('consecutive')
        plt.show()

In [ ]:
print("T(n) = *T(n-1)-0.001")
sizes = [100,500,1000]
distributions = [lambda n : uniform_points(n), lambda n: normal_points(n,4), lambda n: normal_points(n,9)]
for size in sizes:
    for dist in distributions:
        points = dist(size)
        fig,ax =plt.subplots(1,2)
        print(size)
        arb = annealing(points.copy(),arbitrary_swap,lambda x: x-0.001,ax[0])
        con = annealing(points.copy(),consecutive_swap,lambda x: x-0.001,ax[1])
        print("arbitrary swap result:\t\t",length(arb))
        print("consecutive swap result:\t",length(con))
        ax[0].set_title('arbitrary')
        ax[1].set_title('consecutive')
        plt.show()

In [ ]:
print("T(n) = (T(n-1)*np.sin(T(n-1))+99*T(n-1))/100")
for size in sizes:
    for dist in distributions:
        points = dist(size)
        fig,ax =plt.subplots(1,2)
        print(size)
        arb = annealing(points.copy(),arbitrary_swap,lambda x: x*np.sin(x)/100+0.99*x,ax[0])
        con = annealing(points.copy(),consecutive_swap,lambda x: x*np.sin(x)/100+0.99*x,ax[1])
        print("arbitrary swap result:\t\t",length(arb))
        print("consecutive swap result:\t",length(con))
        ax[0].set_title('arbitrary')
        ax[1].set_title('consecutive')
        plt.show()

# Wnioski:
Możemy zauważyć, że w zależności od funkcji temperatury orza sposobu generacji sąsiedniego stanu zmieniała się zbieżność procesu symulowanego wyżarzania. Dla zamian sąsiednich wierzchołków, algorytm radził sobie nieco gorzej niż dla zamian losowych węzłów. Większy wpływ na zbieżność miała za to funkcja temperatury. Najlepiej poradziła sobie funkcja generująca kolejne wyrazy ciągu geometrycznego dla q = 0.99. Nieco gorzej wypadła funkcja generująca kolejne stany temperatury przy użyciu funkcji sinus.
Jednak w jej wypadku powstały stosunkowo duże różnice pomiędzy wynikami uzyskiuwanymi przez consecutive, a arbitrary swap. Najgorzej wypadła funkcja liniowa. Zarówno funkcja wykorzystująca sinus jak i wykładnicza mają początkowo wysoką temperaturę. Moglo mieć to decydujący wpływ na zbieżność wyniku.

# Zadanie 2

In [ ]:
densities = [0.1,0.3,0.4]

In [ ]:
def generate_points(p,n):
    data=np.zeros((n,n),dtype=bool)
    for i in range(n):
        for j in range(n):
            rand = np.random.random()
            if rand >= p:
                data[i][j] = True
    return data

In [ ]:
neighbour_types =[
    [(1,0),(0,1),(0,-1),(-1,0)],
    [(1,0),(0,1),(-1,-1),(1,1)],
    [(1,0),(0,1),(0,-1),(-1,0),(-1,1),(1,-1)],
    [(1,0),(0,1),(0,-1),(-1,0),(1,1),(-1,1),(1,-1),(-1,-1)],
    [(1,0),(0,1),(0,-1),(-1,0),(1,1),(-1,1),(1,-1),(-1,-1),(2,0),(0,2),(0,-2),(-2,0)]
]

In [ ]:
energy_functions = [
    lambda x,y : 1/(x**2+y**2),
    lambda x,y : abs(x)-abs(y),
    lambda x,y : -1/(x**2+y**2),
]

In [ ]:
def similar_state(T,matrix,neighbourhood):
    size = len(matrix)
    swaps = int(size*T)
    for i in range(swaps):
        x = np.random.randint(size)
        y = np.random.randint(size)
        a,b = neighbourhood[np.random.randint(len(neighbourhood))]
        nx,ny = (x+a)%size,(y+b)%size
        tmp = matrix[y,x]
        matrix[y,x] = matrix[ny,nx]
        matrix[ny,nx] = tmp
    return matrix

In [ ]:
def evaluate_energy(matrix,neighbourhood,E_func):
    size = len(matrix)
    E = 0
    for i in range(size):
        for j in range(size):
            for nx, ny in neighbourhood:
                a,b = j + nx, i + ny
                if a < size and a > -1 and b < size and b > -1:
                    sign = 1 if matrix[i][j] == matrix[b][a] else -1
                E += sign*E_func(ny,nx)
    return E

In [ ]:
def equilibrium(points,neighbourhood,E_func,Tfunc = lambda x: x*0.99,ax=None):
    best = points.copy()
    best_energy = evaluate_energy(best,neighbourhood,E_func)
    current_energy = best_energy
    T =0.999
    energies =[best_energy]
    no_change_counter =0
    while int(T*len(points)/5)>0:
        #jeżeli 20 razy nie uzyskam lepszej wartości, to kończę pętlę
        candidate = similar_state(T,copy.deepcopy(points),neighbourhood)
        candidate_energy = evaluate_energy(candidate, neighbourhood, E_func)
        candidate_diff = float(current_energy - candidate_energy)
        if (np.exp((candidate_diff/T)) - np.random.random() > 0):
                points = copy.deepcopy(candidate)
                current_energy = candidate_energy
        energies.append(candidate_energy)
        if candidate_energy< best_energy:
            best_energy = candidate_energy
            best = copy.deepcopy(candidate)
        T = Tfunc(T)
    if ax == None:
        plt.plot([i for i in range(len(energies))],energies)
        plt.show()
    else:
        ax.plot([i for i in range(len(energies))],energies)
    return best

In [ ]:
def make_video(states):
    fig,ax = plt.subplots()
    im = ax.imshow(states[0], animated=True)
    def AnimationFunction(frame):
        im.set_array(frame)
        return im,
        # line is set with new values of x and y
    step = len(states)//60+1
    anim_created = animation.FuncAnimation(fig, AnimationFunction, frames=states[0::step],blit=False)
    video = anim_created.to_html5_video()
    html = display.HTML(video)
    display.display(html)
    plt.show()

In [ ]:
def animate_equilibrium(points,neighbourhood,E_func,Tfunc = lambda x: x*0.99):
    best = points.copy()
    best_energy = evaluate_energy(best,neighbourhood,E_func)
    current_energy = best_energy
    states = [Image.fromarray(best.copy())]
    T =0.999
    energies =[best_energy]
    while int(T*len(points)/5)>0:
        candidate = similar_state(T,copy.deepcopy(points),neighbourhood)
        candidate_energy = evaluate_energy(candidate, neighbourhood, E_func)
        candidate_diff = float(current_energy - candidate_energy)
        if (np.exp((candidate_diff/T)) - np.random.random() > 0):
                points = copy.deepcopy(candidate)
                current_energy = candidate_energy
        energies.append(candidate_energy)
        states.append(Image.fromarray(candidate))
        if candidate_energy< best_energy:
            best_energy = candidate_energy
            best = copy.deepcopy(candidate)
        T = Tfunc(T)
    make_video(states)
    plt.plot([i for i in range(len(energies))],energies)
    plt.show()
    return best

In [ ]:
points = generate_points(0.3,40)
i =0
for n in neighbour_types:
    print("neighbour type: ",(i)//len(energy_functions))
    for ef in energy_functions:
        print("energy function: ",(i)%len(energy_functions))
        fig,ax =plt.subplots(1,3)
        x=equilibrium(points,n,ef,lambda x: x*0.95,ax[0])
        x=equilibrium(points,n,ef,lambda x: x*0.999,ax[1])
        x=equilibrium(points,n,ef,lambda x: x*0.995,ax[2])
        ax[0].set_title('T(n) = 0.95*T(n-1)')
        ax[1].set_title('T(n) = 0.99*T(n-1)')
        ax[2].set_title('T(n) = 0.995*T(n-1)')
        plt.show()
        i+=1
            

# Przykładowe działanie algorytmu dla T(n) = 0.999*T(n-1)

In [ ]:
points = generate_points(0.4,80)
i =0
for n in neighbour_types:
    print("neighbour type: ",(i)//len(energy_functions))
    for ef in energy_functions:
        print("energy function: ",(i)%len(energy_functions))
        x=animate_equilibrium(points,n,ef,lambda x: x*0.999)
        i+=1
            

# Opis:
Mój algorytm generuje stany pośrednie losując odpowiednią (zależną od temperatury) liczbę punktów i dokonując ich zamiany
z punktami należącymi do ich sąsiedztwa Wykorzystuję 3 różne funkcje energii:
1) E ~ 1/r^2
2) E ~ abs(x)-abs(y) -energia asymetryczne zależy od x i y
3) E ~ r^2
Oraz 5 różnych rodzajów sąsiedztwa: czwórkowe, ósemkowe oraz 3 sąsiedztwa asymetryczne
Zbadalem również wyniki dla 3 różnych szybkości spadku temperatury.

# wnioski:
Możemy zauważyć, że im szybciej spadająca funkcja temperatury, tym szybciej otrzymujemy wynik (temperatura szybciej spada do 0). Jednakże wolniej spadające funkcje temperatury pozwalają na uzyskanie dokładniejszego wyniku. Co ciekawsze możemy zaobserwować pewne różne prawidłowości w rozmieszczeniu punktów na płaszczyźnie w zależności od wykorzystanego punktu i sąsiedztwa. W przypadku sąsiedztwa czwórkowego , dla pierwszej funkcji tworzą się ukośne struktury, a dla drugiej poziome. W przypadku większych sąsiedztw również dla trzeciej funkcji tworzą się ciekawe, większe jednokolorowe struktury

# Zadanie 3

In [ ]:
def read_sudoku(filepath):
    with open(filepath,"r") as file:
        lines = file.readlines()
    sudoku = np.zeros((9,9),dtype = np.uint8)
    lines_split = [line.split() for line in lines]
    fixed = set()
    for i in range(9):
        for j in range(9):
            if lines_split[i][j] != 'x':
                fixed.add((i,j))
                sudoku[i,j] = int(lines_split[i][j])
    first_fill(sudoku,fixed)
    return sudoku,fixed

In [ ]:
def first_fill(sudoku,fixed):
    for i in range(3):
        for j in range(3):
            used =np.array([False for i in range(9)],dtype = np.bool_)
            for k in range(3):
                for l in range(3):
                    x,y = i*3+k,j*3+l
                    if (x,y) in fixed:
                        used[sudoku[x,y]-1]=True
            to_insert = []
            for u in range(len(used)):
                if not used[u]:
                    to_insert.append(u+1)
            np.random.shuffle(to_insert)
            z = 0
            for k in range(3):
                for l in range(3):
                    x,y = i*3+k,j*3+l
                    if not sudoku[x,y]:
                        sudoku[x,y] = to_insert[z]
                        z+=1

In [ ]:
def permutate_sudoku(sudoku,fixed,T):
    for i in range(int(T*81)):
        while True:
            blockx = np.random.randint(3)
            blocky = np.random.randint(3)
            x = np.random.randint(3)
            y = np.random.randint(3)
            nx = np.random.randint(3)
            ny = np.random.randint(3)
            x += blockx*3 
            y += blocky*3 
            nx += blockx*3 
            ny += blocky*3 
            if not ((y,x) in fixed or (ny,nx) in fixed) and not(nx == x and ny == y):
                break
        tmp = sudoku[y,x]
        sudoku[y,x] = sudoku[ny,nx]
        sudoku[ny,nx] = tmp
    return sudoku

In [ ]:
def cost_function(sudoku):
    cost = 0
    for i in range(9):
        cost -= len(set(sudoku[i,:]))
        cost -= len(set(sudoku[:,i]))
    return cost

In [ ]:
def solve_sudoku(sudoku,fixed,Tfunc = lambda x:0.999*x):
    best = copy.deepcopy(sudoku)
    best_cost = cost_function(best)
    current_cost = best_cost
    T = 1
    i=0
    costs =[best_cost]
    while T >=0.001:
        candidate = permutate_sudoku(copy.deepcopy(sudoku),fixed,T)
        candidate_cost = cost_function(candidate)
        candidate_diff = float(current_cost - candidate_cost)
        if (np.exp((candidate_diff/T)) - np.random.random() > 0):
                sudoku = copy.deepcopy(candidate)
                current_cost = candidate_cost
        costs.append(candidate_cost)
        if candidate_cost< best_cost:
            best_cost = candidate_cost
            best = copy.deepcopy(candidate)
        T=Tfunc(T)
        if best_cost == -162:
            break
        i+=1
    plt.plot([i for i in range(len(costs))],costs)
    plt.show()
    print("lowest cost:\t",best_cost)
    if(best_cost == -162):
        print("number of iterations:\t",i)
    return best

In [ ]:
#sudoku1 - 42 wypełnione pola
sudoku1,fixed1 = read_sudoku("sudoku1.txt")
x = solve_sudoku(sudoku1,fixed1)
#sudoku2 - 36 wypełnionych pól
sudoku2,fixed2 = read_sudoku("sudoku2.txt")
x = solve_sudoku(sudoku2,fixed2)
#sudoku3 - 30 wypełnionych pól
sudoku3,fixed3 = read_sudoku("sudoku3.txt")
x = solve_sudoku(sudoku3,fixed3)
#sudoku4 - 25 wypełnionych pól
sudoku4,fixed4 = read_sudoku("sudoku4.txt")
x = solve_sudoku(sudoku4,fixed4)
#sudoku5 - 23 wypełnione pola
sudoku5,fixed5 = read_sudoku("sudoku5.txt")
x = solve_sudoku(sudoku5,fixed5)

In [ ]:
def sudoku_result(sudoku,fixed,Tfunc = lambda x: 0.999*x):
    best = copy.deepcopy(sudoku)
    best_cost = cost_function(best)
    current_cost = best_cost
    T = 1
    i=0
    while T*81 >=1:
        candidate = permutate_sudoku(copy.deepcopy(sudoku),fixed,T)
        candidate_cost = cost_function(candidate)
        candidate_diff = float(current_cost - candidate_cost)
        if (np.exp((candidate_diff/T)) - np.random.random() > 0):
                sudoku = copy.deepcopy(candidate)
                current_cost = candidate_cost
        if candidate_cost< best_cost:
            best_cost = candidate_cost
            best = copy.deepcopy(candidate)
        T=Tfunc(T)
        if best_cost == -162:
            break
        i+=1
    if best_cost == -162:
        return i
    return 0

In [ ]:
def test():
    results = np.zeros((5,))
    successes = np.zeros((5,))
    for i in range(5):
        filename = "sudoku"+str(i+1)+".txt"
        sudoku,fixed = read_sudoku(filename)
        for j in range(20):
            result = sudoku_result(sudoku,fixed)
            if result:
                results[i] += result
                successes[i] +=1
    return results,successes

In [ ]:
results = test()

In [ ]:
for i in range(5):
    sudoku_sizes = [55,49,43,36,30,24]
    if results[1][i] > 0:
        avg_iterations = results[0][i]/results[1][i]
        print("sudoku"+str(i+1)+".txt\tsize:\t",sudoku_sizes[i],"\taverage iterations:\t",round(avg_iterations,1))
    else:
        print("sudoku"+str(i+1)+".txt\tsize:\t",sudoku_sizes[i],"\tno sudoku solved:\t",)

# Opis:
Mój algorytm dla zadanego sudoku wypełnia puste pola w taki sposób, aby w każdym z bloków 3x3 danego sudoku znalazły się wszystkie cyfry od 1 do 9. Dzięki temu, aby wygenerować stan sąsiedni, algorytm zamienia komórki jedynie wewnątrz tego samego bloku 3x3.

# Wnioski:
Okazuje się, że algorytm nie był w stanie znaleźć rozwiązań dla wszystkich sudoku. Problematyczne okazały się te, które miały mało wypełnionych pól. W przypadku sudoku o mniejszyh rozmiarach, Najmniej iteracji było potrzebne dla sudoku z 55 wypełnionymi polami. Świadczy to o fakcie, iż algorytm dziala mało efektywnie, gdy sudoku ma wiele wypełnionych pól.